# A Telegram Bot - Connect 4

Hello! This notebook is meant to be a "tutorial-by-doing" for setting up a [Telegram](https://telegram.org/) chat bot, in particular one that allows two people to play Connect 4 in a group chat that the bot is also in.

If you haven't already, I heavily suggest you take a look at the Telegram [Bot API](https://core.telegram.org/bots) as well as the [introduction](https://github.com/python-telegram-bot/python-telegram-bot#introduction) to the [Python Telegram API](https://github.com/python-telegram-bot/python-telegram-bot). Additionally, the steps taken in this notebook heavily follow the tutorial [here](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-%E2%80%93-Your-first-Bot), so be sure to reference that if anything becomes unclear.

For the private values needed throughout this tutorial, I've placed them in a file named `my_env.py`. First we'll pull the bot's access token into local scope.

In [1]:
import my_env as env
my_token = env.connect4_token

From here, I'm just following the tutorial above. First we need import the `telegram` package from the Python API (oh boy did I underestimate how difficult this would be when you have near-zero understanding of how Anaconda and Jupyter Notebooks interact).

You'll need to install the API package from [here](https://github.com/python-telegram-bot/python-telegram-bot/blob/master/README.rst#installing). If you are able to `import telegram` at this point without error, you can skip the next cell. I was not that fortunate.

This Jupyter Notebook was unable to find the package which I had installed to my Anaconda environment (after both the `pip` method and cloning the entire repo). After reading [this](https://github.com/jupyter/notebook/issues/1524) and [this](https://stackoverflow.com/questions/4383571/importing-files-from-different-folder), I came up with the solution in the following cell. It can be summed up as "copying the Anaconda path to this notebook's path."

Since the Python in my Anaconda environment was able to `import telegram`, I knew the path used there was correct. I ran `import sys; sys.path` and copied that list to `my_env.py`. I then add each item to this notebook's path.

In [2]:
# Skip this if the next cell works just fine for you.
path = env.anaconda_path
import sys
for p in path:
    sys.path.insert(0, p)

In [3]:
import telegram

Woo hoo! It worked! Now we can actually follow the tutorial. To start working with the bot, we need to initialize an `Updater` object with the bot's token and pull the appropriate `Dispather` object from there.

In [4]:
from telegram.ext import Updater
updater = Updater(token=my_token)
dispatcher = updater.dispatcher

Now to begin handling each of the commands my bot supports, of which there are 4 explicit ones:
1. `/start_game` - Initiates setup for 2 players 
2. `/p1` - Player who sends this will be designated as Player 1
3. `/p2` - Same as `/p1`, but for Player 2
4. `/quit` - Allows a player to concede or restart setup

Once the game is started, the players can use the custom reply keyboard markup to select which column to place their chips, meaning the bot will also have to handle plain text messages. In order for Telegram to even pass normal text messages to your bot, you need to turn "Group Privacy" off via the BotFather.

To do all this, there's gonna need to be some bookkeeping. Obviously, there also needs to be some way of managing the Connect4 game itself. That will be done through the `Connect4` class in `Connect4.py`.

Something I didn't realize until much later is that method handlers registered with the dispatcher won't be able to access values in this notebook's scope (meaning global variables in this notebook do no good). To "get around" this (as discussed [here](https://github.com/python-telegram-bot/python-telegram-bot/issues/1002)), we can wrap everything in a class and use `self` references to access the "global" values.

In [5]:
# Connect4_Bot.py
import string
from telegram import ReplyKeyboardMarkup, ReplyKeyboardRemove

class Connect4_Bot(object):

    def __init__(self, game):
        ## Bookkeeping Globals ##
        # Universal Values
        self.game            = game
        self.setupHasStarted = False
        self.gameHasStarted  = False
        self.p_cur           = 0
        # Player values
        self.p_set           = [False, False]
        self.p_id            = [0, 0]
        self.p_name          = ['', '']
        # Keyboard Markup
        custom_keyboard = [['1', '2', '3', '4', '5', '6', '7']]
        self.rp_markup  = ReplyKeyboardMarkup(custom_keyboard)
        self.rm_markup  = ReplyKeyboardRemove()

    ### The Commands ###

    # /start_game
    def start_game(self, bot, update):

        chat_id = update.message.chat_id
        text = ''

        # Setup has yet to begin
        if(not(self.setupHasStarted)):
            self.setupHasStarted = True
            text = (r'~~~~ Welcome to Connect 4! ~~~~' + '\n'
                    r'  Player 1, please select /p1' + '\n'
                    r'  Player 2, please select /p2')
        # Setup has begun, but a player still needs to be set
        elif(not(self.gameHasStarted)):
            if(not(self.p_set[0])):
                text = r'Player 1 still needs to be set. Use /p1 to do so.'
            elif(not(self.p_set[1])):
                text = r'Player 2 still needs to be set. Use /p2 to do so.'
        # Game has started
        else:
            text = 'The game has already started silly goose!'

        bot.send_message(chat_id=chat_id, text=text)

    # /p1
    def p1(self, bot, update):

        chat_id = update.message.chat_id
        user = update.message.from_user

        # Player 1 has yet to be set
        if(not(self.setupHasStarted) or 
           (self.setupHasStarted and not(self.p_set[0]))):
            self.setupHasStarted = True
            self.p_set[0]  = True
            self.p_id[0]   = user.id
            self.p_name[0] = user.first_name
            text = self.p_name[0] + r' has been set as Player 1.'
            bot.send_message(chat_id=chat_id, text=text)
        # Player 1 is set but game has not started
        elif(self.p_set[0] and not(self.gameHasStarted)):
            text = self.p_name[0] + r' is already Player 1!'
            bot.send_message(chat_id=chat_id, text=text)
        
        # If both players are set, start the game!
        if(self.p_set[1] and not(self.gameHasStarted)):
            self.start_for_real(bot, chat_id)

    # /p2
    def p2(self, bot, update):

        chat_id = update.message.chat_id
        user = update.message.from_user

        # Player 2 has yet to be set
        if(not(self.setupHasStarted) or 
           (self.setupHasStarted and not(self.p_set[1]))):
            self.setupHasStarted = True
            self.p_set[1]  = True
            self.p_id[1]   = user.id
            self.p_name[1] = user.first_name
            text = self.p_name[1] + r' has been set as Player 2.'
            bot.send_message(chat_id=chat_id, text=text)
        # Player 2 is set but game has not started
        elif(self.p_set[1] and not(self.gameHasStarted)):
            text = self.p_name[1] + r' is already Player 2!'
            bot.send_message(chat_id=chat_id, text=text)
        
        # If both players are set, start the game!
        if(self.p_set[0] and not(self.gameHasStarted)):
            self.start_for_real(bot, chat_id)

    # /quit
    def quit(self, bot, update):

        chat_id = update.message.chat_id
        text = ''

        if(not(self.setupHasStarted)):
            text = (r"You can't quit a game that hasn't even started yet..."
                    '\n' + r'Use /start_game to being setup.')
        elif(not(self.gameHasStarted)):
            text = 'Resetting setup.'
            self.reset_game()
        elif(self.p_id[0] == user_id):
            text = self.p_name[0]+' is a quitter! ' + self.p_name[1]+' wins!'
            self.reset_game()
        elif(self.p_id[1] == user_id):
            text = self.p_name[1]+' is a quitter! ' + self.p_name[0]+' wins!'
            self.reset_game()

        bot.send_message(chat_id=chat_id, text=text, 
                         reply_markup=self.rm_markup)

    ### Command Helpers ###

    def start_for_real(self, bot, chat_id):
        self.gameHasStarted = True
        text = 'Let the games begin!'
        bot.send_message(chat_id=chat_id, text=text)
        text = self.p_name[0] + '\'s turn!' + '\n' + self.game.boardToEmojis()
        bot.send_message(chat_id=chat_id, text=text, 
                         reply_markup=self.rp_markup)

    def reset_game(self):
        self.game.reset()
        self.setupHasStarted = False
        self.gameHasStarted = False
        self.p_cur = 0
        
        self.p_set = [False, False]
        self.p_id = [0, 0]
        self.p_name = ['', '']

    ### Player Actions ###

    def place_chip(self, bot, update):

        text = update.message.text
        chat_id = update.message.chat_id
        user_id = update.message.from_user.id

        if(text in string.digits and int(text) >= 1 and int(text) <= 7):
            if(self.gameHasStarted):
                if((self.p_cur == 1 and not(self.p_id[0] == user_id)) or
                   (self.p_cur == 2 and not(self.p_id[1] == user_id))):
                    text = 'It is not your turn!'
                    bot.send_message(chat_id=chat_id, text=text)
                else:
                    self.handle_move(bot, chat_id, int(text))

    ### Helpers ###

    def next_player(self):
        if(self.p_cur == 0):
            self.p_cur = 1
        elif(self.p_cur == 1):
            self.p_cur = 0

    def handle_move(self, bot, chat_id, col):
        
        res = self.game.placeChip(self.p_cur+1, col)
        
        if(res == -1):
            text = "You can't place a chip there! Try again."
            bot.send_message(chat_id=chat_id, text=text)
        elif(res == 0):
            self.next_player()
            text = (self.p_name[self.p_cur] + '\'s turn!'
                    '\n' + self.game.boardToEmojis())
            bot.send_message(chat_id=chat_id, text=text)
        elif(res == 1):
            text = (self.p_name[self.p_cur] + ' wins!'
                    '\n' + self.game.boardToEmojis())
            self.reset_game()
            bot.send_message(chat_id=chat_id, text=text, 
                             reply_markup=self.rm_markup)
        else:
            text = ('Well... it\'s a tie... good job... I guess.'
                    '\n' + self.game.boardToEmojis())
            self.reset_game()
            bot.send_message(chat_id=chat_id, text=text, 
                             reply_markup=self.rm_markup)


Now that all the commands are ready, we need to register them with the dispatcher. Before we get to that, however, I want to add a filter such that only designated users can interact with my chat bot.

In [6]:
from telegram.ext import BaseFilter
class WhitelistFilter(BaseFilter):
    def __init__(self, whitelist):
        self.whitelist = whitelist

    def filter(self, message):
        return message.from_user.id in self.whitelist
my_filter = WhitelistFilter(env.user_whitelist)

`CommandHandler`s have a `filters` attribute on top of taking the command name and callback method. The first parameter of the `MessageHandler` class takes a filter as well. Note that filters can be combined with binary operators (`&`, `|`, and `~`, which are `and`, `or`, and `not`, respectively).

Now that the whitelist filter is ready, we can register all of the commands with the dispatcher.

In [7]:
# Need to initialize the Connect4 wrapper to add its methods to the handlers
from Connect4 import Connect4
game = Connect4()
my_bot = Connect4_Bot(game)

# Create all handlers for the bot
from telegram.ext import CommandHandler
start_game_handler  = CommandHandler('start_game', my_bot.start_game, 
                                     filters=my_filter)
p1_handler          = CommandHandler('p1', my_bot.p1, filters=my_filter)
p2_handler          = CommandHandler('p2', my_bot.p2, filters=my_filter)
quit_handler        = CommandHandler('quit', my_bot.quit, filters=my_filter)

from telegram.ext import MessageHandler, Filters
# Filters.text means the message must be text only
place_chip_handler = MessageHandler((Filters.text & my_filter), 
                                    my_bot.place_chip)

# Add all of the handlers to the dispatcher
dispatcher.add_handler(start_game_handler)
dispatcher.add_handler(p1_handler)
dispatcher.add_handler(p2_handler)
dispatcher.add_handler(quit_handler)
dispatcher.add_handler(place_chip_handler)

And that's that! Everything should be in place for the bot to function at the basic level. it just needs to run!

In [8]:
# Starts the bot on a separate thread, meaning you have to use `updater.stop()`
# to halt it
updater.start_polling()

In [9]:
# Stops the bot
updater.stop()

There are a few more intricacies and "good habits" concerning proper bot execution, but I'm definitely not one to effectively relay that knowledge. I suggest checking out the [code snippets](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Code-snippets) and Python bot [examples](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Examples) to get a better idea of how your bot should be structured and utilize.

Further note that the `Connect4_Bot` class above is just for quick reference from within this notebook, and the most up-to-date version can be found in `Connect4_Bot.py`. Additionally, this has all been set up to be run via a script, see `bot_script.py` to figure out how to set something like that up.